At the heart of many tensor network algorithms are tensor decomposition and compression operations. These are particularly useful for Matrix Product State (MPS) based algorithms where they are used to compress virtual bonds between tensors. This can lead to a more memory efficient representations and also more efficient compute operations.

## Tensor Decomposition

In PicoQuant, we have implemented a tensor decomposition operation which works by:

1. Reshaping the tensor to matrix
2. Applying matrix decomposition methods (SVD used at present but QR should work also)
3. Applying a cutoff and discarding any singular values and corresponding matrix rows/columns below the given threshold
4. Reshaping the resulting matrices to end up with two tensors connected by a virtual bond

We demonstrate its use by decomposing a two-qubit CNOT gate into two tensors, each acting on a different qubit.

In [ ]:
using PicoQuant

The first thing we do is create a tensor network circuit with two qubits with a single CNOT gate acting on these.

In [ ]:
# we will create an interactive backend so operations will be executed interactively
InteractiveBackend()

# create empty tensor network circuit with 2 qubits
tn = TensorNetworkCircuit(2)

# add a 2 qubit CNOT gate
println("CX dims: $(size(gate_tensor(:CX)))")
add_gate!(tn, gate_tensor(:CX), [1, 2]);

We add input and output nodes so that when we plot the network, it will display the outgoing index labels.

In [ ]:
add_input!(tn, "00")
add_output!(tn, "00")
plot(tn, showlabels=true)

Next we call the `decompose_tensor!` method to decompose the two qubit gate. We pass the tensor network circuit, the symbol for the node we wish to decompose and two arrays of symbols for the sets of indices each of the decomposed tensors will have.

In [ ]:
decompose_tensor!(tn, :node_1, [:index_1, :index_3], [:index_2, :index_4], threshold=0.75)

This method returns the symbols of the decomposed tensors (also possible to provide these as optional arguments).

In [ ]:
size(load_tensor_data(backend, :node_6))

In [ ]:
plot(tn, showlabels=true)

After this we can see that the rank four tensor has been replaced by two rank three tensors connected with a virtual bond. Because it is often the case that one would want to decompose the tensor for two-qubit gates into tensors acting on each individual gate, this decomposition can be done in PicoQuant when adding the gate to the circuit (`add_gate!` method) or creating a tensor network circuit object from a qiskit circuit object (`convert_qiskit_circ_to_network` method) by passing `decompose=true` to these methods. For example, for the circuit above with the single CNOT gate, this would proceed something like. 

In [ ]:
tn = TensorNetworkCircuit(2)
add_input!(tn, "00")
add_output!(tn, "00")
add_gate!(tn, gate_tensor(:CX), [1, 2], decompose=true)
plot(tn, showlabels=true)

## Tensor Network Compression

Compression of a tensor network can be achieved via a sequence of contraction and decomposition operations. There is a caveat that this generally only works for a 1D chain of tensors (for > 2d networks, one can map to a 1D network or contract along the additional dimensions to arrive at a 1d network). 

In PicoQuant, the `compress_tensor_chain!` method compresses a chain of tensors by performing forward and backward passes over the nodes given and compressing the bonds between nodes. The compression of each bond proceeds by:

1. Contracting the two tensors to a single tensor
2. Decomposing the tensor back to two separate tensors using the `decompose_tensor!` method explained above

It is assumed when using this method that the only bonds that exist between tensors in the chain are those between consecutive tensors.

### Chain compression

We illustrate the `compress_tensor_chain!` method with a simple example where we compress a chain of input tensors which have no bonds between them. This should result in bonds of dimension 1 between each tensor.

In [ ]:
# create the tensor network circuit and add 0's for input
tn = TensorNetworkCircuit(3)
add_input!(tn, "000")

Next we look at the nodes and print the size of the tensor for each.

In [ ]:
for node_sym in [:node_1, :node_2, :node_3]
    println("$node_sym dim: $(size(load_tensor_data(backend, node_sym)))")
end

We see that each node is a vector of dimension 2. We now apply compression along the tensor chain.

In [ ]:
# apply compression
compress_tensor_chain!(tn, [:node_1, :node_2, :node_3])

And print the dimension of the resulting tensors.

In [ ]:
for node_sym in [:node_1, :node_2, :node_3]
    println("$node_sym dim: $(size(load_tensor_data(backend, node_sym)))")
end

In [ ]:
println("Nodes in tensor network circuit: $(keys(tn.nodes))")
for node_sym in keys(tn.nodes)
    println("$node_sym dim: $(size(load_tensor_data(backend, node_sym)))")
end

## MPS contraction

Next we show a less trivial example where we contract a 3-qubit tensor network circuit using the `contract_mps_tensor_network_circuit!` method which periodically applied the `compress_tensor_chain!` method to keep the dimension of the bonds down. We use a circuit made up of two applications of a GHZ state preparation circuit to show this.

In [ ]:
# create the qiskit circuit object by combining two 
ghz_circ = create_ghz_preparation_circuit(3)
double_ghz_circ = ghz_circ.compose(ghz_circ)
double_ghz_circ.draw()

In [ ]:
tn = convert_qiskit_circ_to_network(double_ghz_circ, decompose=true)
add_input!(tn, "000")

In [ ]:
plot(tn, showlabels=true)

In [ ]:
mps_nodes = contract_mps_tensor_network_circuit!(tn)

We now print the dimensions of each of the MPS tensors.

In [ ]:
for node_sym in mps_nodes
    println("$node_sym dims: $(size(load_tensor_data(backend, node_sym)))")
end

We see that the bond dimension between each of the input tensors is two. This is a reduction from what the bond dimension would have been if we had contracted without compression (between each consecutive pair of tensors there would be two bonds each of dimension two, see plot above). 

To access the individual amplitudes of the resulting MPS state we can create an instance of the MPSState type. This structure provides an array like interface to the amplitudes which accepts either a configuration string or index.

In [ ]:
mps_state = MPSState(tn, mps_nodes)
println(mps_state["000"])
println(mps_state[1, 1, 1])

Can retrieve a vector from this structure using the vec method

In [ ]:
vec(mps_state)